In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader

import numpy as np 
import pathlib

# Importing utitility functions for training
from PT_files.model import DnCNN, DnCNN_B
from PT_files.Dataset import Img_Dataset, Large_Img_Dataset
import PT_files.preprocess_data as ppd
import PT_files.save_load as sl
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle


device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

Using cuda device


In [ ]:
# data_6k = sl.NERSC_load('test_data70-6000.npy')
data_6k_percent = sl.NERSC_load('test_data_40%_6000.npy')
# full_img_inf = sl.NERSC_load('2k_afterburner_img.npy')
# full_img_inf_adam = sl.NERSC_load('2k_img_inf_adam.npy')
# full_img_inf_sgd = sl.NERSC_load('2k_img_inf_SGD.npy')

# full_sgd = sl.NERSC_load('6k_img_inf_SGD.npy')
full_adam = sl.NERSC_load('6k_img_inf_adam.npy')

# img_inf = sl.NERSC_load('2k_img_inf_WB500.npy')
# WB_6k = sl.NERSC_load('6k_WB_e200.npy')
WB_6k_20 = sl.NERSC_load('6k_WB_e800.npy')
# WB_6k_2048 = sl.NERSC_load('6k_WB_e800_lys20_48feat.npy')
# WB_6k_17 = sl.NERSC_load('6k_WB_e800_lys17.npy')
# WB_6k_15 = sl.NERSC_load('6k_WB_e800_lys15.npy')
# WB_6k_12 = sl.NERSC_load('6k_WB_e800_lys12.npy')
# WB_6k_2052 = sl.NERSC_load('6k_WB_e800_lys20_52feat.npy')
WB_6k_2058 = sl.NERSC_load('6k_WB_e800_lys20_58feat.npy')
# WB_6k_1952= sl.NERSC_load('6k_WB_e400_lys19_52feat.npy')
# WB_6k_1032 = sl.NERSC_load('6k_model_wb_e800_lys10_32feat.npy')
numpy_denoised = sl.NERSC_load('first_numpy_denoise.npy')

In [ ]:
# DENOISED IMAGE IS DIFFERENT THAN ALL THE OTHERS FIX

def plot_comparison(raw_data,
                    inf_data,
                    size,
                    start_idx,
                    end_idx):
    
    data_6k = raw_data
    full_img_inf = inf_data
    
    assert size == end_idx-start_idx
    
    
    fig, ax = plt.subplots(1, 4, figsize=(24,20))
    vmin, vmax = np.percentile(data_6k[0][0][0][start_idx:end_idx,start_idx:end_idx], (1,99))


    ax[0].imshow(data_6k[0][0][0], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    # ax[0,0].axis('off')
    ax[0].set_title('Original Image', fontsize=30)
    ax[0].axis('off')
    rect = Rectangle((1600, 1600), 200, 200, linewidth=4, edgecolor='r', facecolor='none')
    ax[0].add_patch(rect)
    ax[1].imshow(data_6k[0][0][0][start_idx:end_idx,start_idx:end_idx], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    # ax[0,1].axis('off')
    ax[1].set_title('Noisy Sub-Image', fontsize=30)
    ax[1].axis('off')
    ax[2].imshow(full_img_inf[0][0][start_idx:end_idx,start_idx:end_idx],vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[2].axis('off')
    ax[2].set_title('Denoised Sub-Image', fontsize=30)
    ax[3].imshow(data_6k[1][0][0][start_idx:end_idx,start_idx:end_idx], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[3].axis('off')
    ax[3].set_title('Truth Sub-Image', fontsize=30)

plot_comparison(raw_data=data_6k_percent,
                             inf_data=WB_6k_20,
                             size=200,
                             start_idx=1400,
                             end_idx=1600)

In [ ]:
# Create figure and axes
fig, ax = plt.subplots()

# Display the image
ax.imshow(data_6k_percent[0][0][0][0:1800, 0:1800], origin='lower')

# Create a Rectangle patch
rect = Rectangle((1600, 1600), 200, 200, linewidth=3, edgecolor='r', facecolor='none')

# Add the patch to the Axes
ax.add_patch(rect)

plt.show()

In [ ]:
plt.imshow(data_6k_percent[0][0][0][:1600, :1600], origin='lower')
plt.Rectangle((1600, 1600), 200, 200, linewidth=4, edgecolor='r', facecolor='none')


In [ ]:
def pytorch_numpy_comparison(raw_data,
                             pytorch_inf,
                             numpy_inf,
                             size,
                             start_idx,
                             end_idx):
    
    data_6k = raw_data
    full_img_inf = pytorch_inf
    
    assert size == end_idx-start_idx
    
    
    fig, ax = plt.subplots(1, 4, figsize=(28,24))
    vmin, vmax = np.percentile(data_6k[0][0][0][start_idx:end_idx,start_idx:end_idx], (1,99))


    ax[0].imshow(data_6k[0][0][0][start_idx:end_idx,start_idx:end_idx], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[0].axis('off')
    ax[0].set_title('Noisy Sample', fontsize=30)
    ax[1].imshow(full_img_inf[0][0][start_idx:end_idx,start_idx:end_idx],vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[1].axis('off')
    ax[1].set_title('Pytorch Sample', fontsize=30)
    ax[2].imshow(numpy_inf[0][0][start_idx:end_idx,start_idx:end_idx],vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[2].axis('off')
    ax[2].set_title('Numpy Sample', fontsize=30)
    ax[3].imshow(data_6k[1][0][0][start_idx:end_idx,start_idx:end_idx], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[3].axis('off')
    ax[3].set_title('Truth Sample', fontsize=30)

In [ ]:
# pytorch_numpy_comparison(raw_data=data_6k_percent,
#                              pytorch_inf=full_adam,
#                              numpy_inf=numpy_denoised,
#                              size=200,
#                              start_idx=2400,
#                              end_idx=2600)

In [ ]:
def plot_comparison(raw_data,
                    inf_data,
                    size,
                    start_idx,
                    end_idx):
    
    data_6k = raw_data
    full_img_inf = inf_data
    
    assert size == end_idx-start_idx
    
    
    fig, ax = plt.subplots(1, 3, figsize=(24,20))
    vmin, vmax = np.percentile(data_6k[0][0][0][start_idx:end_idx,start_idx:end_idx], (1,99))


    ax[0].imshow(data_6k[0][0][0][start_idx:end_idx,start_idx:end_idx], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[0].axis('off')
    ax[0].set_title('Noisy Sample', fontsize=30)
    ax[1].imshow(full_img_inf[0][0][start_idx:end_idx,start_idx:end_idx],vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[1].axis('off')
    ax[1].set_title('Denoised Sample', fontsize=30)
    ax[2].imshow(data_6k[1][0][0][start_idx:end_idx,start_idx:end_idx], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[2].axis('off')
    ax[2].set_title('Truth Sample', fontsize=30)
    
    
def model_plot_comparison(i, raw_data, size, start_idx, end_idx):
    
    data_6k = raw_data
    inf_img_data = [WB_6k_20, WB_6k_2058, WB_6k_2052, WB_6k_17, WB_6k_15, WB_6k_12]
    noisy_title = ["Noisy: 20lys 64feat", "Noisy: 20lys 58feat","Noisy: 19lys 52feat", "Noisy Sample 17lys",
                   "Noisy Sample 15lys", "Noisy Sample 12lys"]
    denoised_title = ["Denoised: 20lys 64feat","Denoised: 20lys 58feat", "Denoised: 19lys 52feat", "Denoised Sample 17lys",
                      "Denoised Sample 15lys", "Denoised Sample 12lys"]
    truth_title = ["Truth: 20lys 64feat", "Truth: 20lys 58feat", "Truth: 19lys 52feat", "Truth Sample 17lys",
                   "Truth Sample 15lys", "Truth Sample 12lys"]
    # inf_img_data = [WB_6k_20, WB_6k_2052]
    # noisy_title = ["Noisy: 20lys 64feat", "Noisy: 20lys 52feat"]
    # denoised_title = ["Denoised: 20lys 64feat", "Denoised: 20lys 52feat"]
    # truth_title = ["Truth: 20lys 64feat", "Truth: 20lys 52feat"]


    
    assert size == end_idx-start_idx
    
    
    fig, ax = plt.subplots(1, 3, figsize=(24,20))
    vmin, vmax = np.percentile(data_6k[0][0][0][start_idx:end_idx,start_idx:end_idx], (1,99))

        
    ax[0].imshow(data_6k[0][0][0][start_idx:end_idx,start_idx:end_idx], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[0].axis('off')
    ax[0].set_title(noisy_title[i], fontsize=30)
    ax[1].imshow(inf_img_data[i][0][0][start_idx:end_idx,start_idx:end_idx],vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[1].axis('off')
    ax[1].set_title(denoised_title[i], fontsize=30)
    ax[2].imshow(data_6k[1][0][0][start_idx:end_idx,start_idx:end_idx], vmin=vmin, vmax=vmax, origin='lower', interpolation='none')
    ax[2].axis('off')
    ax[2].set_title(truth_title[i], fontsize=30)
    plt.show()
    

In [ ]:
# WB_6k_20 = sl.NERSC_load('6k_WB_e800.npy')
# WB_6k_2048 = sl.NERSC_load('6k_WB_e800_lys20_48feat.npy')
# WB_6k_17 = sl.NERSC_load('6k_WB_e800_lys17.npy')
# WB_6k_15 = sl.NERSC_load('6k_WB_e800_lys15.npy')
# WB_6k_12 = sl.NERSC_load('6k_WB_e800_lys12.npy')
# WB_6k_2052 = sl.NERSC_load('6k_WB_e800_lys20_52feat.npy')
# WB_6k_2058 = sl.NERSC_load('6k_WB_e800_lys20_58feat.npy')
# WB_6k_1952= sl.NERSC_load('6k_WB_e400_lys19_52feat.npy')

In [ ]:
# for i in range(7):
#     model_plot_comparison(i,
#                           raw_data=data_6k_percent,
#                           size=200,
#                           start_idx=1400,
#                           end_idx=1600)